In [ ]:
#Inital Model
# Step 1: Install dependencies
# Run this in your terminal
# pip install transformers torch accelerate

from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

# Step 2: Load the model and tokenizer
def load_model(model_name_or_path="meta-llama/CodeLlama-7b-hf"):
    """
    Load Code Llama model and tokenizer from Hugging Face.

    Args:
        model_name_or_path (str): Path or name of the model.

    Returns:
        model, tokenizer: Loaded model and tokenizer.
    """
    tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
    model = LlamaForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
    return model, tokenizer

# Step 3: Inference
if __name__ == "__main__":
    # Load the model and tokenizer
    model, tokenizer = load_model()

    # Example input prompt
    input_prompt = "# Write a Python function to calculate the factorial of a number"

    # Tokenize input
    inputs = tokenizer(input_prompt, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    # Decode output tokens
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(output_text)

In [ ]:
#Code Generation
# Step 1: Install dependencies
# Run this in your terminal
# pip install transformers torch accelerate

from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

# Step 2: Load the model and tokenizer
def load_model(model_name_or_path="meta-llama/CodeLlama-7b-hf"):
    """
    Load Code Llama model and tokenizer from Hugging Face.

    Args:
        model_name_or_path (str): Path or name of the model.

    Returns:
        model, tokenizer: Loaded model and tokenizer.
    """
    tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
    model = LlamaForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
    return model, tokenizer

# Step 3: Inference
if __name__ == "__main__":
    # Load the model and tokenizer
    model, tokenizer = load_model()

    # Example input prompt
    input_prompt = "# Write a Python function to calculate the factorial of a number"

    # Tokenize input
    inputs = tokenizer(input_prompt, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    # Decode output tokens
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(output_text)

In [ ]:
#Code Debugging

# Step 1: Install dependencies
# Run this in your terminal
# pip install transformers torch accelerate

from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

# Step 2: Load the model and tokenizer
def load_model(model_name_or_path="meta-llama/CodeLlama-7b-hf"):
    """
    Load Code Llama model and tokenizer from Hugging Face.

    Args:
        model_name_or_path (str): Path or name of the model.

    Returns:
        model, tokenizer: Loaded model and tokenizer.
    """
    tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
    model = LlamaForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
    return model, tokenizer

# Step 3: Code debugging with Code Llama
if __name__ == "__main__":
    # Load the model and tokenizer
    model, tokenizer = load_model()

    # Example input prompt for code debugging
    input_prompt = """
# The following Python code has a bug. Fix the code:

def find_max_value(lst):
    max_val = lst[0]
    for i in range(1, len(lst)):
    if lst[i] > max_val:
        max_val = lst[i]
    return max_val
"""

    # Tokenize input
    try:
        inputs = tokenizer(input_prompt, return_tensors="pt").to(model.device)
    except Exception as e:
        print(f"Error during tokenization: {e}")
        exit(1)

    # Generate code output
    try:
        with torch.no_grad():
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=150,  # Increase max tokens for more complex code
                temperature=0.2,  # Lower temperature for more deterministic code
                top_p=0.9,  # Adjust top_p to balance creativity and reliability
                do_sample=False  # Set to False for deterministic output
            )
    except Exception as e:
        print(f"Error during code generation: {e}")
        exit(1)

    # Decode output tokens
    try:
        output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        print("Suggested Fix:\n")
        print(output_text)
    except Exception as e:
        print(f"Error during decoding: {e}")
        exit(1)

In [ ]:
# Step 1: Install dependencies
# Run this in your terminal
# pip install transformers torch accelerate

from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

# Step 2: Load the model and tokenizer
def load_model(model_name_or_path="meta-llama/CodeLlama-7b-hf"):
    """
    Load Code Llama model and tokenizer from Hugging Face.

    Args:
        model_name_or_path (str): Path or name of the model.

    Returns:
        model, tokenizer: Loaded model and tokenizer.
    """
    tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
    model = LlamaForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
    return model, tokenizer

# Step 3: Generate documentation with Code Llama
if __name__ == "__main__":
    # Load the model and tokenizer
    model, tokenizer = load_model()

    # Example input prompt for writing documentation
    input_prompt = """
# Write detailed documentation for the following Python function:

def merge_sorted_lists(list1, list2):
    """
    Merges two sorted lists into a single sorted list.

    Args:
        list1 (list): The first sorted list.
        list2 (list): The second sorted list.

    Returns:
        list: A merged and sorted list containing all elements from list1 and list2.
    """
    merged_list = []
    i = j = 0
    while i < len(list1) and j < len(list2):
        if list1[i] < list2[j]:
            merged_list.append(list1[i])
            i += 1
        else:
            merged_list.append(list2[j])
            j += 1
    merged_list.extend(list1[i:])
    merged_list.extend(list2[j:])
    return merged_list
"""

    # Tokenize input
    try:
        inputs = tokenizer(input_prompt, return_tensors="pt").to(model.device)
    except Exception as e:
        print(f"Error during tokenization: {e}")
        exit(1)

    # Generate documentation output
    try:
        with torch.no_grad():
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=150,  # Increase max tokens for detailed documentation
                temperature=0.2,  # Lower temperature for more deterministic output
                top_p=0.9,  # Adjust top_p to balance creativity and reliability
                do_sample=False  # Set to False for deterministic output
            )
    except Exception as e:
        print(f"Error during documentation generation: {e}")
        exit(1)

    # Decode output tokens
    try:
        output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        print("Generated Documentation:\n")
        print(output_text)
    except Exception as e:
        print(f"Error during decoding: {e}")
        exit(1)
